# Monolith To MicroServices Udacity Project

## Setup Secrets

Copy the [sample.set_env.sh](./sample.set_env.sh) into a new file [set_env.sh](./set_env.sh.

Edit the secrets to the appropriate values.

In [1]:
!source set_env.sh